In [34]:
import pandas as pd
import numpy as np
import random
import json
import time
import datetime
import os

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

import torch
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup

In [35]:
ADDITIONAL_SPECIAL_TOKENS = ["<e1>", "</e1>", "<e2>", "</e2>"]

In [36]:
data_fname = './data/train_relation_candidates.csv'
df = pd.read_csv(data_fname)
print('train data size: ', len(df))
df.head()

train data size:  7187


,abstract_id,sentences,entity_1_id,entity_2_id,entity_1_mention,entity_2_mention,type,kfold
0,19707748,Drug-drug interactions related to the inhibiti...,D064704,1544,levofloxacin,CYP1A2,Association,0
1,24914936,Thyroid hormone receptor a mutation causes a s...,D013974,D001848,thyroxine,skeletal dysplasia,Association,0
2,28101574,Alteration of NEK2 protein levels may contribu...,4130,207,MMP-2,AKT,Negative,0
3,30836660,Further study revealed that the effect of Sal ...,"21898,7099","26413,5594",TLR4,MAPK,Negative,0
4,27491646,Gartanin induces cell cycle arrest and autopha...,5291,2475,PI3K,mTOR,Association,0


In [37]:
df.type.value_counts()

Association             2514
Negative                2263
Positive_Correlation    1287
Negative_Correlation     894
Comparison                89
Bind                      70
Cotreatment               53
Drug_Interaction          13
Conversion                 4
Name: type, dtype: int64

In [38]:
label_map_dic = {
    'Association': 0,
    'Positive_Correlation': 1,
    'Negative_Correlation': 2,
    'Bind': 3,
    'Cotreatment': 4,
    'Comparison':5,
    'Drug_Interaction':6,
    'Conversion':7,
    'Negative':8
      }
df['label'] = df['type'].map(label_map_dic)
df.head()

,abstract_id,sentences,entity_1_id,entity_2_id,entity_1_mention,entity_2_mention,type,kfold,label
0,19707748,Drug-drug interactions related to the inhibiti...,D064704,1544,levofloxacin,CYP1A2,Association,0,0
1,24914936,Thyroid hormone receptor a mutation causes a s...,D013974,D001848,thyroxine,skeletal dysplasia,Association,0,0
2,28101574,Alteration of NEK2 protein levels may contribu...,4130,207,MMP-2,AKT,Negative,0,8
3,30836660,Further study revealed that the effect of Sal ...,"21898,7099","26413,5594",TLR4,MAPK,Negative,0,8
4,27491646,Gartanin induces cell cycle arrest and autopha...,5291,2475,PI3K,mTOR,Association,0,0


In [39]:
# Instantiate the Bert tokenizer
# biobert 
pretrained_fname = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = BertTokenizer.from_pretrained(pretrained_fname, do_lower_case=False)
tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})
print('information about tokenizer: \n', tokenizer)

information about tokenizer: 
 PreTrainedTokenizer(name_or_path='dmis-lab/biobert-base-cased-v1.1', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>']})


In [40]:
def get_inputs(df):
    input_ids = []
    attention_masks = []
    labels = []
    #e_mask = []
    e1_mask_list = []
    e2_mask_list = []

    MAX_LENGTH = 128

    # For every sentence...
    for i,row in df.iterrows():
        sent = row['sentences'] + str('[SEP]') + row['entity_1_mention'] + str('[SEP]') + row['entity_2_mention']
        #sent = sent[:124]
        labels.append(int(row['label']))
        
        encoded_dict = tokenizer.encode_plus(
                          sent,                          # Sentence to encode.
                          add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                          max_length = MAX_LENGTH,       # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,  # Construct attn. masks.
                          return_tensors = 'pt',         # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
        #print(encoded_dict['attention_mask'])
        # e1 mask, e2 mask
        e1_mask = torch.zeros_like(encoded_dict['attention_mask'])
        e2_mask = torch.zeros_like(encoded_dict['attention_mask'])
        
        tokens_a = tokenizer.tokenize(sent)
        e11_p = tokens_a.index('<e1>')  # the start position of entity1
        e12_p = tokens_a.index('</e1>')  # the end position of entity1
        e21_p = tokens_a.index('<e2>')  # the start position of entity2
        e22_p = tokens_a.index('</e2>')  # the end position of entity2
        for i in range(e11_p, e12_p + 1):
            if i < 128:
                e1_mask[0][i] = 1
            else:
                continue
        for i in range(e21_p, e22_p + 1):
            if i < 128:
                e2_mask[0][i] = 1
            else:
                continue
        #print(e1_mask)
        e1_mask_list.append(e1_mask)
        e2_mask_list.append(e2_mask)
        
    #print(attention_masks)
    #print(e1_mask_list)
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    e1_mask_list = torch.cat(e1_mask_list, dim=0)
    e2_mask_list = torch.cat(e2_mask_list, dim=0)
    
    return input_ids,attention_masks,labels,e1_mask_list,e2_mask_list

In [41]:
kfold = 0
train_df = df[df.kfold != kfold].reset_index(drop=True)
valid_df = df[df.kfold == kfold].reset_index(drop=True)  

train_input_ids, train_attention_masks, train_labels, train_e1_mask, train_e2_mask = get_inputs(train_df)
valid_input_ids, valid_attention_masks, valid_labels, valid_e1_mask, valid_e2_mask = get_inputs(valid_df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [42]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels,train_e1_mask, train_e2_mask)
val_dataset = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels,valid_e1_mask, valid_e2_mask)

In [43]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [44]:
model = BertForSequenceClassification.from_pretrained(
               'dmis-lab/biobert-base-cased-v1.1', 
               num_labels = len(label_map_dic), # The number of output labels. 2 for binary classification.  
               output_attentions = False,
               output_hidden_states = False)   

# after add special token, we should resize model size
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# send model to device
model.to(device);

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [45]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 10

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps = 0, # Default value in run_glue.py
                       num_training_steps = total_steps)

In [31]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and retur`````
    ns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [46]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# store a number of quantities such as training and validation loss,validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # training
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader andcopy each tensor to the GPU
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing abackward pass.
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask, 
                labels=b_labels)

        # Accumulate the training loss over all of the batches
        loss= outputs[0]
        logits = outputs[1]  
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    # ========================================
    # Save model of every epoch
    output_dir = './outputs/biobert_models/'
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    models_dir = output_dir + 'model_epoch_' + str(epoch_i + 1)
    # Create output directory if needed
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    print("Saving model to %s" % models_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(models_dir)
    tokenizer.save_pretrained(models_dir)
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0.0
    total_eval_loss = 0.0
    nb_eval_steps = 0.0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs= model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
            
        # Accumulate the validation loss.
        loss= outputs[0]
        logits = outputs[1] 
        
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 10 ========
Training...
  Batch    40  of    180.    Elapsed: 0:00:09.
  Batch    80  of    180.    Elapsed: 0:00:18.
  Batch   120  of    180.    Elapsed: 0:00:27.
  Batch   160  of    180.    Elapsed: 0:00:36.

  Average training loss: 1.0883
  Training epoch took: 0:00:41
Saving model to ./outputs/biobert_models/model_epoch_1

Running Validation...
  Accuracy: 0.7371
  Validation Loss: 0.7731
  Validation took: 0:00:03
======== Epoch 2 / 10 ========
Training...
  Batch    40  of    180.    Elapsed: 0:00:09.
  Batch    80  of    180.    Elapsed: 0:00:18.
  Batch   120  of    180.    Elapsed: 0:00:27.
  Batch   160  of    180.    Elapsed: 0:00:36.

  Average training loss: 0.6482
  Training epoch took: 0:00:41
Saving model to ./outputs/biobert_models/model_epoch_2

Running Validation...
  Accuracy: 0.7747
  Validation Loss: 0.6604
  Validation took: 0:00:03
======== Epoch 3 / 10 ========
Training...
  Batch    40  of    180.    Elapsed: 0:00:09.
  Batch    80  of   

In [48]:
import os
# Display floats with two decimal places.
pd.set_option('precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

output_dir = './outputs/biobert_models/'
# # Create output directory if needed
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

df_stats.to_csv(output_dir + 'df_train_stats.csv',index=None)

# according df_stats, pick the best performace model
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.0883,0.7731,0.7371,0:00:41,0:00:03
2,0.6482,0.6604,0.7747,0:00:41,0:00:03
3,0.4028,0.6902,0.7933,0:00:41,0:00:03
4,0.2682,0.7430,0.7892,0:00:41,0:00:03
5,0.1764,0.8015,0.8024,0:00:41,0:00:03
6,0.1254,0.8769,0.8073,0:00:41,0:00:03
7,0.0838,0.9444,0.8156,0:00:41,0:00:03
8,0.0675,0.9557,0.8143,0:00:41,0:00:03
9,0.0476,1.0164,0.8142,0:00:41,0:00:03
